In [1]:
from pathlib import Path
from PIL import Image
import numpy as np

def compute_minmax(hr_dir, lr_dir, max_files_per_folder=None):
    hr_dir = Path(hr_dir)
    lr_dir = Path(lr_dir)

    mins = []
    maxs = []

    def process_folder(folder):
        tiffs = sorted(folder.rglob("*.tiff"))
        if max_files_per_folder is not None:
            tiffs = tiffs[:max_files_per_folder]
        for p in tiffs:
            with Image.open(p) as im:
                arr = np.array(im, dtype=np.float32)
            mins.append(arr.min())
            maxs.append(arr.max())

    process_folder(hr_dir)
    process_folder(lr_dir)

    vmin = float(np.min(mins))
    vmax = float(np.max(maxs))
    return vmin, vmax

# пример использования:
hr_path = "MRCCM2D/HR_train"
lr_path = "MRCCM2D/LR_train"

vmin, vmax = compute_minmax(hr_path, lr_path)
print("vmin:", vmin, "vmax:", vmax)

vmin: 8557.25 vmax: 47033.0


In [3]:
from pathlib import Path
from PIL import Image
import numpy as np

def compute_percentiles(hr_dir,
                        lr_dir,
                        p_low=0.5,
                        p_high=99.5,
                        max_files_per_folder=None,
                        downsample=None):
    """
    Считает p_low и p_high перцентили по всем пикселям из HR и LR.
    
    hr_dir, lr_dir: пути к папкам с .tif
    p_low, p_high: перцентили (в процентах)
    max_files_per_folder: если задать число, возьмёт только первые N файлов из каждой папки
    downsample: если задать число k, будет брать каждый k-й пиксель (по строкам и столбцам)
    """
    hr_dir = Path(hr_dir)
    lr_dir = Path(lr_dir)

    vals = []

    def process_folder(folder: Path):
        tiffs = sorted(folder.rglob("*.tiff"))
        if max_files_per_folder is not None:
            tiffs = tiffs[:max_files_per_folder]
        for p in tiffs:
            with Image.open(p) as im:
                arr = np.array(im, dtype=np.float32)
            if downsample is not None and downsample > 1:
                arr = arr[::downsample, ::downsample]
            vals.append(arr.ravel())

    process_folder(hr_dir)
    process_folder(lr_dir)

    all_vals = np.concatenate(vals)
    vmin = float(np.percentile(all_vals, p_low))
    vmax = float(np.percentile(all_vals, p_high))
    return vmin, vmax

# пример использования
hr_path = "MRCCM2D/HR_train"
lr_path = "MRCCM2D/LR_train"

vmin, vmax = compute_percentiles(
    hr_path,
    lr_path,
    p_low=0.5,
    p_high=99.5,
    max_files_per_folder=100,  # можно None, если файлов немного
    downsample=None              # можно None, если памяти хватает
)

print("p0.5:", vmin)
print("p99.5:", vmax)

p0.5: 11072.5
p99.5: 18645.0
